In [38]:
import ndcctools.taskvine as vine
import json
import os

In [40]:
%%bash
time python bin/CalcDatasetSize.py --directory data/parquet/3split

Retrying with http://inaturalist-open-data.s3.amazonaws.com/photos/2632365/large.JPG...
Retrying with http://inaturalist-open-data.s3.amazonaws.com/photos/1619491/large.JPG...
Retrying with http://inaturalist-open-data.s3.amazonaws.com/photos/3578342/large.JPG...
Retrying with http://inaturalist-open-data.s3.amazonaws.com/photos/269390774/large.JPG...
Retrying with http://inaturalist-open-data.s3.amazonaws.com/photos/855018/large.JPG...
Retrying with http://inaturalist-open-data.s3.amazonaws.com/photos/3748764/large.JPG...
Retrying with http://inaturalist-open-data.s3.amazonaws.com/photos/3748739/large.JPG...
Retrying with http://inaturalist-open-data.s3.amazonaws.com/photos/855042/large.JPG...
Retrying with http://inaturalist-open-data.s3.amazonaws.com/photos/344439301/large.JPG...
Retrying with http://inaturalist-open-data.s3.amazonaws.com/photos/3554977/large.JPG...
Retrying with http://inaturalist-open-data.s3.amazonaws.com/photos/1612253/large.JPG...
Retrying with http://inaturali


real	0m30.779s
user	0m5.351s
sys	0m1.033s


In [41]:
%%bash
time python bin/MakeTaskvineSlurm.py

My local IP addresses are: 


Traceback (most recent call last):
  File "/Users/zideng/Work/LDAWT/bin/MakeTaskvineSlurm.py", line 49, in <module>
    main()
  File "/Users/zideng/Work/LDAWT/bin/MakeTaskvineSlurm.py", line 18, in main
    print(local_ips.split()[0])
          ~~~~~~~~~~~~~~~~~^^^
IndexError: list index out of range

real	0m0.039s
user	0m0.018s
sys	0m0.009s


CalledProcessError: Command 'b'time python bin/MakeTaskvineSlurm.py\n'' returned non-zero exit status 1.

In [24]:
%%bash
sbatch taskvine.slurm
python bin/TaskvineLDAWT.py 

Submitted batch job 3432095
Listening on port 9124
Waiting for tasks to complete...
